In [1]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from math import exp
import io
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

In [2]:
filename = "iris.data.txt"
r = io.open(filename, encoding='utf8').readlines()
X = []
Y = []
for i in r[0:150]:
    x = i.split(',')
    X.append(map(float,x[0:len(x)-1]))
    if x[-1] == "Iris-setosa\n":
        Y.append(0)
    elif x[-1] == 'Iris-versicolor\n':
        Y.append(1)
    elif x[-1] == 'Iris-virginica\n':
        Y.append(2)
Y = np.array(Y)

In [3]:
#to determine hidden layer weights we must have no. of features + 1 columumns
for i in X:
    i.insert(0,1)
X = np.array(X)

In [4]:
def hiddenWeights(h,x):
    w = np.random.rand(h,x.shape[1])/100000
    return w

In [5]:
def findZ(w,x):
    z = np.ones([len(x),len(w)+1])
    for i in range(len(w)):
        for j in range(len(x)):
            z[j][i+1] = np.dot(w[i].T,x[j])
            
    return z

In [6]:
def outputWeights(h,y):
    k = len(np.unique(y))
    v = np.random.rand(k,h+1)/100000
    return v
            

In [7]:
def ycap(v,z):
    yc = [[0]*len(v)]*len(z)
    for i in range(len(z)):
        de = 0
        for j in range(len(v)):
            de = de + np.exp(np.dot(v[j].T,z[i]))
        for k in range(len(v)):
            nu = np.exp(np.dot(v[k].T,z[i]))
            yc[i][k] = nu/de
    return np.array(yc)

In [8]:
def find_indicator(yy):
    k = np.unique(yy)
    ind = np.zeros([len(yy),len(k)])
    for i in range(len(k)):
        for j in range(len(yy)):
            if k[i] == yy[j]:
                ind[j][i] = 1
    return ind
            

In [9]:
def descent(data,y,h,rate,count):
    W = hiddenWeights(h,data)
    V = outputWeights(h,y)
    indy = find_indicator(y)
    for i in range(count):
        Z = findZ(W,data)
        yh = ycap(V,Z)
        tmp = yh-indy
        for k in range(tmp.shape[1]):
            sumz = 0
            for j in range(len(Z)):
                sumz = sumz + tmp[j][k]*Z[j]
            V[k] = V[k] - rate*sumz
        for m in range(len(W)):
            sumw = 0
            for n in range(len(data)):
                sumv = 0 
                for l in range(tmp.shape[1]):
                    sumv = sumv + tmp[n][l]*V[l][m+1]
                sumw = sumw + (sumv*Z[n][m+1]*(1-Z[n][m+1])*data[n])
            W[m] = W[m] - rate*sumw
    return V,W

In [10]:
def classify(v,w,data):
    p = []
    zc = findZ(w,data)
    yc = ycap(v,zc)
    for i in yc:
        p.append([np.argmax(i)])
    return np.array(p)

In [11]:
def accuracy(cm):
    return np.trace(cm)*1.0/np.sum(cm)

In [14]:
def confusionmatrix(y,classifier_pred):
    clabels = [0,1,2]
    cm_fold = []
    for i in clabels:
        tmp =[0]*len(clabels)
        for j in range(len(y)):
            #print j
            if y[j] == i and y[j] == classifier_pred[j]:
                tmp[clabels.index(i)] += 1
            elif y[j] == i and y[j] != classifier_pred[j]:
                tmp[clabels.index(classifier_pred[j])] += 1
        cm_fold.append(tmp)
    return cm_fold

In [15]:
Vf,Wf = descent(X,Y,4,0.00008451,800)
classified = classify(Vf,Wf,X)
np.unique(classified)

array([0], dtype=int64)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
Vf,Wf = descent(X_train,y_train,4,0.00008451,800)
classified = classify(Vf,Wf,X_test)
np.unique(classified)

cm = confusionmatrix(y_test,classified)
acc = accuracy(cm)

print cm
print acc


[[0, 19, 0], [0, 15, 0], [0, 16, 0]]
0.3
